# Homework 4

## 2. Grouping songs together!

### The final goal is to group songs into similar genres!

__Libraries:__

In [1]:
import pandas as pd
import numpy as np
import librosa

You are supposed to create one single data set by merging __tracks.csv, features.csv and echonest.csv__. It's your job to find the correct key to join your data sets together. You should end up with a data set of ~13K rows.

First of all I will import using pandas the csv file.

In [2]:
echonest = pd.read_csv('echonest.csv')

In [3]:
features = pd.read_csv('features.csv')

In [4]:
tracks = pd.read_csv('tracks.csv')

Let's get an info on the data:

In [5]:
df_list = [echonest, features, tracks]

for df in df_list:
    print(df.shape)

(13129, 250)
(106574, 519)
(106574, 53)


Hanno un numero di colonne diverse, vediamo se hanno qualcuna in comune:

In [6]:
col = []

for df in df_list:
    col.append(set(df.columns))

In [7]:
common_labels = set.intersection(*col)
common_labels

{'track_id'}

Possiamo notare che features and tracks hanno lo stesso numero di righe, presuppongo che quindi le righe dei due dataframe hanno
lo stesso track id e sono collegate

In [8]:
echonest.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,...,temporal_features_214,temporal_features_215,temporal_features_216,temporal_features_217,temporal_features_218,temporal_features_219,temporal_features_220,temporal_features_221,temporal_features_222,temporal_features_223
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,NaN,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,NaN,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,NaN,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008-03-11,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,NaN,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705


In [9]:
features.head()

,track_id,chroma_cens_kurtosis_01,chroma_cens_kurtosis_02,chroma_cens_kurtosis_03,chroma_cens_kurtosis_04,chroma_cens_kurtosis_05,chroma_cens_kurtosis_06,chroma_cens_kurtosis_07,chroma_cens_kurtosis_08,chroma_cens_kurtosis_09,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [10]:
tracks.head()

,track_id,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_id,album_information,album_listens,album_producer,...,track_information,track_interest,track_language_code,track_license,track_listens,track_lyricist,track_number,track_publisher,track_tags,track_title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


Osservazione:

* devo avere un dataset di 13k rows --> solo le track di echonest!


In [11]:
dataset = pd.concat([echonest, features, tracks], axis=1, join = 'inner')

In [12]:
dataset = dataset.loc[:,~dataset.columns.duplicated()]  # removing duplicates columns

In [13]:
dataset.shape

(13129, 820)

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
dataset.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,metadata_album_name,metadata_artist_latitude,metadata_artist_location,metadata_artist_longitude,metadata_artist_name,metadata_release,ranks_artist_discovery_rank,ranks_artist_familiarity_rank,ranks_artist_hotttnesss_rank,ranks_song_currency_rank,ranks_song_hotttnesss_rank,social_features_artist_discovery,social_features_artist_familiarity,social_features_artist_hotttnesss,social_features_song_currency,social_features_song_hotttnesss,temporal_features_000,temporal_features_001,temporal_features_002,temporal_features_003,temporal_features_004,temporal_features_005,temporal_features_006,temporal_features_007,temporal_features_008,temporal_features_009,temporal_features_010,temporal_features_011,temporal_features_012,temporal_features_013,temporal_features_014,temporal_features_015,temporal_features_016,temporal_features_017,temporal_features_018,temporal_features_019,temporal_features_020,temporal_features_021,temporal_features_022,temporal_features_023,temporal_features_024,temporal_features_025,temporal_features_026,temporal_features_027,temporal_features_028,temporal_features_029,temporal_features_030,temporal_features_031,temporal_features_032,temporal_features_033,temporal_features_034,temporal_features_035,temporal_features_036,temporal_features_037,temporal_features_038,temporal_features_039,temporal_features_040,temporal_features_041,temporal_features_042,temporal_features_043,temporal_features_044,temporal_features_045,temporal_features_046,temporal_features_047,temporal_features_048,temporal_features_049,temporal_features_050,temporal_features_051,temporal_features_052,temporal_features_053,temporal_features_054,temporal_features_055,temporal_features_056,temporal_features_057,temporal_features_058,temporal_features_059,temporal_features_060,temporal_features_061,temporal_features_062,temporal_features_063,temporal_features_064,temporal_features_065,temporal_features_066,temporal_features_067,temporal_features_068,temporal_features_069,temporal_features_070,temporal_features_071,temporal_features_072,temporal_features_073,temporal_features_074,temporal_features_075,temporal_features_076,temporal_features_077,temporal_features_078,temporal_features_079,temporal_features_080,temporal_features_081,temporal_features_082,temporal_features_083,temporal_features_084,temporal_features_085,temporal_features_086,temporal_features_087,temporal_features_088,temporal_features_089,temporal_features_090,temporal_features_091,temporal_features_092,temporal_features_093,temporal_features_094,temporal_features_095,temporal_features_096,temporal_features_097,temporal_features_098,temporal_features_099,temporal_features_100,temporal_features_101,temporal_features_102,temporal_features_103,temporal_features_104,temporal_features_105,temporal_features_106,temporal_features_107,temporal_features_108,temporal_features_109,temporal_features_110,temporal_features_111,temporal_features_112,temporal_features_113,temporal_features_114,temporal_features_115,temporal_features_116,temporal_features_117,temporal_features_118,temporal_features_119,temporal_features_120,temporal_features_121,temporal_features_122,temporal_features_123,temporal_features_124,temporal_features_125,temporal_features_126,temporal_features_127,temporal_features_128,temporal_features_129,temporal_features_130,temporal_features_131,temporal_features_132,temporal_features_133,temporal_features_134,temporal_features_135,temporal_features_136,temporal_features_137,temporal_features_138,temporal_features_139,temporal_features_140,temporal_features_141,temporal_features_142,temporal_features_143,temporal_features_144,temporal_features_145,temporal_features_146,temporal_features_147,temporal_features_148,temporal_features_149,temporal_features_150,tempor

In [16]:
dataset.album_type.unique()

array(['Album', 'Single Tracks', 'Live Performance', nan, 'Radio Program'],
      dtype=object)

In [17]:
duplicates = ['track_title','metadata_album_name', 'album_title', 'metadata_album_date', 'album_date_created', 'album_date_released'
             , 'track_date_created', 'track_date_recorded']
dataset.loc[dataset.album_type == 'Single Tracks', duplicates].head()

,track_title,metadata_album_name,album_title,metadata_album_date,album_date_created,album_date_released,track_date_created,track_date_recorded
10,Father's Day,NaN,mp3,NaN,2008-11-26 01:49:19,2009-01-07 00:00:00,2008-11-26 01:43:26,2008-11-26 00:00:00
11,Peel Back The Mountain Sky,NaN,mp3,NaN,2008-11-26 01:49:19,2009-01-07 00:00:00,2008-11-26 01:43:35,2008-11-26 00:00:00
29,Maps of the Stars Homes,NaN,unreleased demo,NaN,2008-11-26 01:51:00,2009-01-06 00:00:00,2008-11-26 01:45:10,2008-11-26 00:00:00
43,Boss of Goth,NaN,Boss of Goth,NaN,2008-11-26 01:51:48,2008-06-23 00:00:00,2008-11-26 01:46:02,2006-01-01 00:00:00
44,Industry Standard Massacre,NaN,Nightmarcher,NaN,2008-11-26 01:51:50,2007-01-01 00:00:00,2008-11-26 01:46:05,2006-10-07 00:00:00


In [18]:
dataset.loc[dataset.album_type == 'Album', duplicates].head()

,track_title,metadata_album_name,album_title,metadata_album_date,album_date_created,album_date_released,track_date_created,track_date_recorded
0,Food,NaN,AWOL - A Way Of Life,NaN,2008-11-26 01:44:45,2009-01-05 00:00:00,2008-11-26 01:48:12,2008-11-26 00:00:00
1,Electric Ave,NaN,AWOL - A Way Of Life,NaN,2008-11-26 01:44:45,2009-01-05 00:00:00,2008-11-26 01:48:14,2008-11-26 00:00:00
2,This World,NaN,AWOL - A Way Of Life,NaN,2008-11-26 01:44:45,2009-01-05 00:00:00,2008-11-26 01:48:20,2008-11-26 00:00:00
3,Freeway,Constant Hitmaker,Constant Hitmaker,2008-03-11,2008-11-26 01:45:08,2008-02-06 00:00:00,2008-11-25 17:49:06,2008-11-26 00:00:00
4,Spiritual Level,NaN,Niris,NaN,2008-11-26 01:45:05,2009-01-06 00:00:00,2008-11-26 01:48:56,2008-01-01 00:00:00


Remove all the columns with NaN value:

In [19]:
NaN_columns = dataset.isna().sum() / dataset.shape[0] 

In [20]:
NaN_columns[NaN_columns > 0].nsmallest(10)

artist_date_created         0.001295
track_language_code         0.002742
track_license               0.003961
album_title                 0.014396
album_date_created          0.026430
album_type                  0.032523
album_information           0.211745
album_date_released         0.229263
artist_website              0.246401
metadata_artist_latitude    0.255846
dtype: float64

In [21]:
Missing50 = NaN_columns[NaN_columns > 50/100]  # 50% dei dati mancanti
print(Missing50)
print("\nMinimums percentage:\n",Missing50.nsmallest(3))

metadata_album_date              0.793054
metadata_album_name              0.781248
ranks_artist_discovery_rank      0.784827
ranks_artist_familiarity_rank    0.784827
ranks_artist_hotttnesss_rank     0.784904
ranks_song_currency_rank         0.845152
ranks_song_hotttnesss_rank       0.831975
album_engineer                   0.685201
album_producer                   0.665854
artist_active_year_begin         0.653591
artist_active_year_end           0.912179
artist_associated_labels         0.746287
artist_related_projects          0.764415
artist_wikipedia_page            0.869373
track_composer                   0.950796
track_date_recorded              0.656943
track_information                0.909666
track_lyricist                   0.994287
track_publisher                  0.989108
dtype: float64

Minimums percentage:
 artist_active_year_begin    0.653591
track_date_recorded         0.656943
album_producer              0.665854
dtype: float64


In [22]:
Missing50 = list(Missing50.index.values)
Missing50

['metadata_album_date',
 'metadata_album_name',
 'ranks_artist_discovery_rank',
 'ranks_artist_familiarity_rank',
 'ranks_artist_hotttnesss_rank',
 'ranks_song_currency_rank',
 'ranks_song_hotttnesss_rank',
 'album_engineer',
 'album_producer',
 'artist_active_year_begin',
 'artist_active_year_end',
 'artist_associated_labels',
 'artist_related_projects',
 'artist_wikipedia_page',
 'track_composer',
 'track_date_recorded',
 'track_information',
 'track_lyricist',
 'track_publisher']

Removing empty columns:

In [23]:
Empty = (dataset == '[]' ).sum(axis = 0) / dataset.shape[0] 
EmptyCols = Empty[Empty > 0]

In [24]:
EmptyCols

album_tags          0.918349
artist_tags         0.005332
track_genres        0.015538
track_genres_all    0.015538
track_tags          0.916064
dtype: float64

In [25]:
EmptyCols = list(Empty[Empty > 90/100].index.values) # only the 90% of missing data
EmptyCols

['album_tags', 'track_tags']

Removing columns we cannot use:

In [26]:
genre_features = ['track_genres_all', 'track_genres', 'track_genre_top' ]
dataset.loc[:, genre_features].head()

,track_genres_all,track_genres,track_genre_top
0,[21],[21],Hip-Hop
1,[21],[21],Hip-Hop
2,[21],[21],Hip-Hop
3,[10],[10],Pop
4,"[17, 10, 76, 103]","[76, 103]",NaN


There are one column that the creator of the dataset use for let us do classification, we can't use it.

In [27]:
split_dataset = ['set_split', 'set_subset']
print(dataset.set_split.unique())

['training' 'validation' 'test']


In [28]:
dataset.loc[:, split_dataset].head()

,set_split,set_subset
0,training,small
1,training,medium
2,training,small
3,training,small
4,training,large


Informazioni non utili per raggruppere sui genere

In [29]:
ToDrop = ['album_title', 'artist_date_created', 'artist_website', 'album_date_created', 'album_date_released',
          'album_information','artist_bio','artist_members','track_number','track_title','track_date_created',
          'set_split','set_subset']

dataset.loc[:, ToDrop].head()

,album_title,artist_date_created,artist_website,album_date_created,album_date_released,album_information,artist_bio,artist_members,track_number,track_title,track_date_created,set_split,set_subset
0,AWOL - A Way Of Life,2008-11-26 01:42:32,http://www.AzillionRecords.blogspot.com,2008-11-26 01:44:45,2009-01-05 00:00:00,<p></p>,"<p>A Way Of Life, A Collective of Hip-Hop from...","Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",3,Food,2008-11-26 01:48:12,training,small
1,AWOL - A Way Of Life,2008-11-26 01:42:32,http://www.AzillionRecords.blogspot.com,2008-11-26 01:44:45,2009-01-05 00:00:00,<p></p>,"<p>A Way Of Life, A Collective of Hip-Hop from...","Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",4,Electric Ave,2008-11-26 01:48:14,training,medium
2,AWOL - A Way Of Life,2008-11-26 01:42:32,http://www.AzillionRecords.blogspot.com,2008-11-26 01:44:45,2009-01-05 00:00:00,<p></p>,"<p>A Way Of Life, A Collective of Hip-Hop from...","Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",6,This World,2008-11-26 01:48:20,training,small
3,Constant Hitmaker,2008-11-26 01:42:55,http://kurtvile.com,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,"<p><span style=""font-family:Verdana, Geneva, A...","Kurt Vile, the Violators",1,Freeway,2008-11-25 17:49:06,training,small
4,Niris,2008-11-26 01:42:52,NaN,2008-11-26 01:45:05,2009-01-06 00:00:00,"<p> ""spiritual songs"" from Nicky Cook</p>",<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,Nicky Cook\n,3,Spiritual Level,2008-11-26 01:48:56,training,large


The location part

In [30]:
dataset.loc[:, ['artist_location','metadata_artist_location', 
                'artist_latitude', 'artist_longitude','metadata_artist_latitude', 'metadata_artist_longitude']]

,artist_location,metadata_artist_location,artist_latitude,artist_longitude,metadata_artist_latitude,metadata_artist_longitude
0,New Jersey,"Georgia, US",40.058324,-74.405661,32.6783,-83.22300
1,New Jersey,"Georgia, US",40.058324,-74.405661,32.6783,-83.22300
2,New Jersey,"Georgia, US",40.058324,-74.405661,32.6783,-83.22300
3,NaN,"Philadelphia, PA, US",NaN,NaN,39.9523,-75.16240
4,Colchester England,"Georgia, US",51.895927,0.891874,32.6783,-83.22300
...,...,...,...,...,...,...
13124,NaN,Netherlands,NaN,NaN,52.1082,5.32986
13125,NaN,Netherlands,NaN,NaN,52.1082,5.32986
13126,Kiel & Cologne / Germany,Netherlands,50.910995,6.990187,52.1082,5.32986
13127,NaN,Netherlands,NaN,NaN,52.1082,5.32986


In [31]:
dropLocation = ['metadata_artist_location', 
                'artist_latitude', 'artist_longitude','metadata_artist_latitude', 'metadata_artist_longitude']

Roba inserita male male

In [ ]:
dataset.metadata_artist_location.unique()

In [33]:
dataset.track_language_code.unique()

array(['en', 'es', nan, 'fi', 'pt', 'tr', 'sw', 'el', 'ar', 'pl', 'id',
       'tw', 'eu', 'ms', 'fr', 'ty', 'hy', 'hi', 'vi', 'ja', 'tl', 'it',
       'my', 'gu', 'zh', 'uk', 'az', 'sr', 'lt', 'th', 'bg', 'de', 'no',
       'ko', 'uz', 'ka', 'ha', 'sk', 'nl', 'bm'], dtype=object)

Osservazioni: 

In [34]:
artist_drop = ['artist_id','artist_name','artist_tags']

In [35]:
dataset.loc[:,artist_drop].head()

,artist_id,artist_name,artist_tags
0,1,AWOL,['awol']
1,1,AWOL,['awol']
2,1,AWOL,['awol']
3,6,Kurt Vile,"['philly', 'kurt vile']"
4,4,Nicky Cook,"['instrumentals', 'experimental pop', 'post pu..."


## Drop columns part:

In [105]:
dataset_clean = dataset.copy()

In [106]:
dataset_clean = dataset_clean.drop(columns = Missing50 )
dataset_clean = dataset_clean.drop(columns = EmptyCols )
dataset_clean = dataset_clean.drop(columns = genre_features )
dataset_clean = dataset_clean.drop(columns = ToDrop)
dataset_clean = dataset_clean.drop(columns = dropLocation)
dataset_clean = dataset_clean.drop(columns = artist_drop)

In [107]:
dataset_clean.shape, dataset.shape

((13129, 775), (13129, 820))

In [108]:
# nan values for dataset
sum(dataset.isnull().sum())

251951

In [109]:
# nan values for dataset_clean
sum(dataset_clean.isnull().sum())

5399

In [110]:
# NaN columns to visualize
nanList = dataset_clean.isnull().sum()
nanList[nanList > 0] / dataset_clean.shape[0] * 100

album_type              3.252342
artist_location        37.200091
track_language_code     0.274202
track_license           0.396070
dtype: float64

In [111]:
ToFill = list(nanList[nanList > 0].index.values)
ToFill

['album_type', 'artist_location', 'track_language_code', 'track_license']

Fill with "N/A" the NaN value of the remaining columns

In [112]:
dataset_clean.loc[:, ToFill] = dataset_clean.loc[:, ToFill].fillna('N/A')

In [113]:
sum(dataset_clean.isnull().sum())

0

No missing values.

In [114]:
dataset_clean.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_artist_name,metadata_release,social_features_artist_discovery,social_features_artist_familiarity,social_features_artist_hotttnesss,social_features_song_currency,social_features_song_hotttnesss,temporal_features_000,temporal_features_001,temporal_features_002,temporal_features_003,temporal_features_004,temporal_features_005,temporal_features_006,temporal_features_007,temporal_features_008,temporal_features_009,temporal_features_010,temporal_features_011,temporal_features_012,temporal_features_013,temporal_features_014,temporal_features_015,temporal_features_016,temporal_features_017,temporal_features_018,temporal_features_019,temporal_features_020,temporal_features_021,temporal_features_022,temporal_features_023,temporal_features_024,temporal_features_025,temporal_features_026,temporal_features_027,temporal_features_028,temporal_features_029,temporal_features_030,temporal_features_031,temporal_features_032,temporal_features_033,temporal_features_034,temporal_features_035,temporal_features_036,temporal_features_037,temporal_features_038,temporal_features_039,temporal_features_040,temporal_features_041,temporal_features_042,temporal_features_043,temporal_features_044,temporal_features_045,temporal_features_046,temporal_features_047,temporal_features_048,temporal_features_049,temporal_features_050,temporal_features_051,temporal_features_052,temporal_features_053,temporal_features_054,temporal_features_055,temporal_features_056,temporal_features_057,temporal_features_058,temporal_features_059,temporal_features_060,temporal_features_061,temporal_features_062,temporal_features_063,temporal_features_064,temporal_features_065,temporal_features_066,temporal_features_067,temporal_features_068,temporal_features_069,temporal_features_070,temporal_features_071,temporal_features_072,temporal_features_073,temporal_features_074,temporal_features_075,temporal_features_076,temporal_features_077,temporal_features_078,temporal_features_079,temporal_features_080,temporal_features_081,temporal_features_082,temporal_features_083,temporal_features_084,temporal_features_085,temporal_features_086,temporal_features_087,temporal_features_088,temporal_features_089,temporal_features_090,temporal_features_091,temporal_features_092,temporal_features_093,temporal_features_094,temporal_features_095,temporal_features_096,temporal_features_097,temporal_features_098,temporal_features_099,temporal_features_100,temporal_features_101,temporal_features_102,temporal_features_103,temporal_features_104,temporal_features_105,temporal_features_106,temporal_features_107,temporal_features_108,temporal_features_109,temporal_features_110,temporal_features_111,temporal_features_112,temporal_features_113,temporal_features_114,temporal_features_115,temporal_features_116,temporal_features_117,temporal_features_118,temporal_features_119,temporal_features_120,temporal_features_121,temporal_features_122,temporal_features_123,temporal_features_124,temporal_features_125,temporal_features_126,temporal_features_127,temporal_features_128,temporal_features_129,temporal_features_130,temporal_features_131,temporal_features_132,temporal_features_133,temporal_features_134,temporal_features_135,temporal_features_136,temporal_features_137,temporal_features_138,temporal_features_139,temporal_features_140,temporal_features_141,temporal_features_142,temporal_features_143,temporal_features_144,temporal_features_145,temporal_features_146,temporal_features_147,temporal_features_148,temporal_features_149,temporal_features_150,temporal_features_151,temporal_features_152,temporal_features_153,temporal_features_154,temporal_features_155,temporal_features_156,temporal_features_157,temporal_features_158,temporal_features_159,temporal_features_160,temporal_features_161,temporal_features_1

In [115]:
dataset_clean.shape

(13129, 775)

Creating the csv

In [80]:
dataset_clean.to_csv('dataset.csv', index = False)

In [116]:
language = dataset.track_language_code.unique()

In [ ]:
parse_lang = dict(zip(language, range(len(language))))
parse_lang

In [123]:
for i in range(dataset_clean.shape[0]):
    dataset_clean.loc[i, "track_language_code"] = vocabolary[dataset_clean.track_language_code.loc[i]]

In [124]:
cols = dataset_clean.columns

In [132]:
int(np.where(cols == "zcr_std_01")[0])

767

In [135]:
PCAcols = cols[:int(np.where(cols == "zcr_std_01")[0])+1]

In [158]:
dataset_PCA = dataset_clean.loc[:, PCAcols]

In [159]:
dataset_PCA.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,metadata_album_name,metadata_artist_latitude,metadata_artist_location,metadata_artist_longitude,metadata_artist_name,metadata_release,ranks_artist_discovery_rank,ranks_artist_familiarity_rank,ranks_artist_hotttnesss_rank,ranks_song_currency_rank,ranks_song_hotttnesss_rank,social_features_artist_discovery,social_features_artist_familiarity,social_features_artist_hotttnesss,social_features_song_currency,social_features_song_hotttnesss,temporal_features_000,temporal_features_001,temporal_features_002,temporal_features_003,temporal_features_004,temporal_features_005,temporal_features_006,temporal_features_007,temporal_features_008,temporal_features_009,temporal_features_010,temporal_features_011,temporal_features_012,temporal_features_013,temporal_features_014,temporal_features_015,temporal_features_016,temporal_features_017,temporal_features_018,temporal_features_019,temporal_features_020,temporal_features_021,temporal_features_022,temporal_features_023,temporal_features_024,temporal_features_025,temporal_features_026,temporal_features_027,temporal_features_028,temporal_features_029,temporal_features_030,temporal_features_031,temporal_features_032,temporal_features_033,temporal_features_034,temporal_features_035,temporal_features_036,temporal_features_037,temporal_features_038,temporal_features_039,temporal_features_040,temporal_features_041,temporal_features_042,temporal_features_043,temporal_features_044,temporal_features_045,temporal_features_046,temporal_features_047,temporal_features_048,temporal_features_049,temporal_features_050,temporal_features_051,temporal_features_052,temporal_features_053,temporal_features_054,temporal_features_055,temporal_features_056,temporal_features_057,temporal_features_058,temporal_features_059,temporal_features_060,temporal_features_061,temporal_features_062,temporal_features_063,temporal_features_064,temporal_features_065,temporal_features_066,temporal_features_067,temporal_features_068,temporal_features_069,temporal_features_070,temporal_features_071,temporal_features_072,temporal_features_073,temporal_features_074,temporal_features_075,temporal_features_076,temporal_features_077,temporal_features_078,temporal_features_079,temporal_features_080,temporal_features_081,temporal_features_082,temporal_features_083,temporal_features_084,temporal_features_085,temporal_features_086,temporal_features_087,temporal_features_088,temporal_features_089,temporal_features_090,temporal_features_091,temporal_features_092,temporal_features_093,temporal_features_094,temporal_features_095,temporal_features_096,temporal_features_097,temporal_features_098,temporal_features_099,temporal_features_100,temporal_features_101,temporal_features_102,temporal_features_103,temporal_features_104,temporal_features_105,temporal_features_106,temporal_features_107,temporal_features_108,temporal_features_109,temporal_features_110,temporal_features_111,temporal_features_112,temporal_features_113,temporal_features_114,temporal_features_115,temporal_features_116,temporal_features_117,temporal_features_118,temporal_features_119,temporal_features_120,temporal_features_121,temporal_features_122,temporal_features_123,temporal_features_124,temporal_features_125,temporal_features_126,temporal_features_127,temporal_features_128,temporal_features_129,temporal_features_130,temporal_features_131,temporal_features_132,temporal_features_133,temporal_features_134,temporal_features_135,temporal_features_136,temporal_features_137,temporal_features_138,temporal_features_139,temporal_features_140,temporal_features_141,temporal_features_142,temporal_features_143,temporal_features_144,temporal_features_145,temporal_features_146,temporal_features_147,temporal_features_148,temporal_features_149,temporal_features_150,tempor

In [160]:
toAdd = ['album_comments','album_favorites','album_id','album_listens','album_tracks',
'artist_comments','artist_favorites','track_bit_rate','track_comments','track_duration',
'track_favorites','track_interest',
'track_language_code','track_listens']

In [161]:
dataset_PCA = pd.concat([dataset_PCA, dataset_clean.loc[:, toAdd]], axis = 1)

In [162]:
dataset_PCA.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,metadata_album_name,metadata_artist_latitude,metadata_artist_location,metadata_artist_longitude,metadata_artist_name,metadata_release,ranks_artist_discovery_rank,ranks_artist_familiarity_rank,ranks_artist_hotttnesss_rank,ranks_song_currency_rank,ranks_song_hotttnesss_rank,social_features_artist_discovery,social_features_artist_familiarity,social_features_artist_hotttnesss,social_features_song_currency,social_features_song_hotttnesss,temporal_features_000,temporal_features_001,temporal_features_002,temporal_features_003,temporal_features_004,temporal_features_005,temporal_features_006,temporal_features_007,temporal_features_008,temporal_features_009,temporal_features_010,temporal_features_011,temporal_features_012,temporal_features_013,temporal_features_014,temporal_features_015,temporal_features_016,temporal_features_017,temporal_features_018,temporal_features_019,temporal_features_020,temporal_features_021,temporal_features_022,temporal_features_023,temporal_features_024,temporal_features_025,temporal_features_026,temporal_features_027,temporal_features_028,temporal_features_029,temporal_features_030,temporal_features_031,temporal_features_032,temporal_features_033,temporal_features_034,temporal_features_035,temporal_features_036,temporal_features_037,temporal_features_038,temporal_features_039,temporal_features_040,temporal_features_041,temporal_features_042,temporal_features_043,temporal_features_044,temporal_features_045,temporal_features_046,temporal_features_047,temporal_features_048,temporal_features_049,temporal_features_050,temporal_features_051,temporal_features_052,temporal_features_053,temporal_features_054,temporal_features_055,temporal_features_056,temporal_features_057,temporal_features_058,temporal_features_059,temporal_features_060,temporal_features_061,temporal_features_062,temporal_features_063,temporal_features_064,temporal_features_065,temporal_features_066,temporal_features_067,temporal_features_068,temporal_features_069,temporal_features_070,temporal_features_071,temporal_features_072,temporal_features_073,temporal_features_074,temporal_features_075,temporal_features_076,temporal_features_077,temporal_features_078,temporal_features_079,temporal_features_080,temporal_features_081,temporal_features_082,temporal_features_083,temporal_features_084,temporal_features_085,temporal_features_086,temporal_features_087,temporal_features_088,temporal_features_089,temporal_features_090,temporal_features_091,temporal_features_092,temporal_features_093,temporal_features_094,temporal_features_095,temporal_features_096,temporal_features_097,temporal_features_098,temporal_features_099,temporal_features_100,temporal_features_101,temporal_features_102,temporal_features_103,temporal_features_104,temporal_features_105,temporal_features_106,temporal_features_107,temporal_features_108,temporal_features_109,temporal_features_110,temporal_features_111,temporal_features_112,temporal_features_113,temporal_features_114,temporal_features_115,temporal_features_116,temporal_features_117,temporal_features_118,temporal_features_119,temporal_features_120,temporal_features_121,temporal_features_122,temporal_features_123,temporal_features_124,temporal_features_125,temporal_features_126,temporal_features_127,temporal_features_128,temporal_features_129,temporal_features_130,temporal_features_131,temporal_features_132,temporal_features_133,temporal_features_134,temporal_features_135,temporal_features_136,temporal_features_137,temporal_features_138,temporal_features_139,temporal_features_140,temporal_features_141,temporal_features_142,temporal_features_143,temporal_features_144,temporal_features_145,temporal_features_146,temporal_features_147,temporal_features_148,temporal_features_149,temporal_features_150,tempor

In [163]:
dataset_PCA.to_csv('datasetPCA.csv', index = False)